In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import cv2
from matplotlib import pyplot as plt

/home/sachin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
images=[]
for img in glob.glob("/home/sachin/ML_training/Dog_breed/resized_images/*.jpg"):
    images.append(img)
images.sort()#Returns images list in ascending order
images=images[:10000]

In [3]:
labels_df=pd.read_csv("/home/sachin/ML_training/Dog_breed/labels.csv")
labels_df.sort_values('id', axis=0, ascending=True,inplace=True)

In [4]:

labels_df=labels_df[:10000]
print(labels_df.shape)

(10000, 2)


In [5]:
print(len(images))
images[:10]

10000


['/home/sachin/ML_training/Dog_breed/resized_images/000bec180eb18c7604dcecc8fe0dba07.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/001513dfcb2ffafc82cccf4d8bbaba97.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/001cdf01b096e06d78e9e5112d419397.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/00214f311d5d2247d5dfe4fe24b2303d.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/0021f9ceb3235effd7fcde7f7538ed62.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/002211c81b498ef88e1b40b9abf84e1d.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/00290d3e1fdd27226ba27a8ce248ce85.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/002a283a315af96eaea0e28e7163b21b.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/003df8b8a8b05244b1d920bb6cf451f9.jpg',
 '/home/sachin/ML_training/Dog_breed/resized_images/0042188c895a2f14ef64a918ed9c7b64.jpg']

In [6]:
labels_df.drop(['id'],axis=1,inplace=True)

In [7]:
label_no=list(range(1,121))
label_names=list(labels_df.breed.unique())

In [8]:
label_dict=dict(zip(label_names,label_no))

In [9]:
labels=tf.constant(labels_df.breed.map(label_dict))
labels=tf.one_hot(labels,120)
filename=tf.constant(images)

In [10]:
def parse_function(filename, label):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    resized_image = tf.image.resize_images(image, [224, 224])
    return resized_image, label


In [11]:
def train_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)

    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

In [12]:
dataset=tf.data.Dataset.from_tensor_slices((images, labels))

In [13]:
dataset=dataset.shuffle(len(images))
dataset=dataset.map(parse_function, num_parallel_calls=4)
#dataset=dataset.map(train_preprocess, num_parallel_calls=4)
dataset=dataset.batch(50)
dataset=dataset.repeat(10)
init = tf.global_variables_initializer()
iterator = dataset.make_one_shot_iterator()
x,y = iterator.get_next()

In [14]:
conv1=tf.layers.conv2d(x,filters=96,kernel_size=(11,11),strides=4,padding="SAME",activation=tf.nn.relu,name="conv1")
#pool1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name="pool1")
lrn1 = tf.nn.lrn(input=conv1, depth_radius=5, bias=1.0, alpha=0.0001/5.0, beta=0.75); #Normalization layer
pool1_conv1 = tf.layers.max_pooling2d(inputs=lrn1, pool_size=[3, 3], strides=2) #Max Pool Layer


# conv2=tf.layers.conv2d(conv1,filters=32,kernel_size=3,strides=2,padding="SAME",activation=tf.nn.relu,name="conv2")
# pool2=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name="pool2")
# conv3=tf.layers.conv2d(pool2,filters=8,kernel_size=3,strides=1,padding="SAME",activation=tf.nn.relu,name="conv3")
# conv4=tf.layers.conv2d(conv3,filters=8,kernel_size=3,strides=2,padding="SAME",activation=tf.nn.relu,name="conv4")
# pool4=tf.nn.max_pool(conv4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name="pool4")

In [16]:


conv2 = tf.layers.conv2d(inputs=pool1_conv1, filters=256, kernel_size=[5, 5], strides=1, padding="same", activation=tf.nn.relu)
lrn2 = tf.nn.lrn(input=conv2, depth_radius=5, bias=1.0, alpha=0.0001/5.0, beta=0.75); #Normalization layer
pool2_conv2 = tf.layers.max_pooling2d(inputs=lrn2, pool_size=[3, 3], strides=2) #Max Pool Layer
#print(pool2_conv2)


conv3 = tf.layers.conv2d(inputs=pool2_conv2, filters=384, kernel_size=[3, 3], strides=1, padding="same", activation=tf.nn.relu)
#print(conv3)


conv4 = tf.layers.conv2d(inputs=conv3, filters=384, kernel_size=[3, 3], strides=1, padding="same", activation=tf.nn.relu)
#print(conv4)


conv5 = tf.layers.conv2d(inputs=conv4, filters=256, kernel_size=[3, 3], strides=1, padding="same", activation=tf.nn.relu)
pool3_conv5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[3, 3], strides=2, padding="valid") #Max Pool Layer
#print(pool3_conv5)


#FULLY CONNECTED LAYER 1
#fully-connected layers have 4096 neurons each
pool3_conv5_flat = tf.reshape(pool3_conv5, [-1, 6* 6 * 256]) #output of conv block is 6x6x256 therefore, to connect it to a fully connected layer, we can flaten it out
fc1 = tf.layers.dense(inputs=pool3_conv5_flat, units=4096, activation=tf.nn.relu)
#fc1 = tf.layers.conv2d(inputs=pool3_conv5, filters=4096, kernel_size=[6, 6], strides=1, padding="valid", activation=tf.nn.relu) #representing the FCL using a convolution block (no need to do 'pool3_conv5_flat' above)


#FULLY CONNECTED LAYER 2
"""since the output from above is [1x1x4096]"""
fc2 = tf.layers.dense(inputs=fc1, units=4096, activation=tf.nn.relu)
#fc2 = tf.layers.conv2d(inputs=fc1, filters=4096, kernel_size=[1, 1], strides=1, padding="valid", activation=tf.nn.relu)



In [17]:

logits = tf.layers.dense(inputs=fc2, units=120, name="logits_layer")
#fc3 = tf.layers.conv2d(inputs=fc2, filters=43, kernel_size=[1, 1], strides=1, padding="valid")
#logits = tf.layers.dense(inputs=fc3, units=120) 
#print(logits)

In [18]:
y_prob=tf.nn.softmax(logits,name="y_prob")

In [ ]:
# pool4_flat=tf.contrib.layers.flatten(pool4)
# logits=tf.layers.dense(pool4_flat,120,name="output")
# y_prob=tf.nn.softmax(logits,name="y_prob")

In [19]:
#Layers/Archi of our tensorflow model
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
loss=tf.reduce_mean(cross_entropy)
optimizer=tf.train.AdamOptimizer(learning_rate=0.0001)
training_op=optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


init = tf.global_variables_initializer()
# saver = tf.train.Saver()


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [20]:
with tf.Session() as sess:
    sess.run(init)
    epoch=0

    for batch_no in range(1,2001):
        if(batch_no%200==0):
            epoch=epoch+1      
        loss_val,accuracy_val,_=sess.run([loss,accuracy,training_op])

        print("Epoch: ",epoch,"Batch_no: ",batch_no,"Loss :===",loss_val,"     ","Accuracy:==",accuracy_val)
        
    

Epoch:  0 Batch_no:  1 Loss :=== 13.572389       Accuracy:== 0.0
Epoch:  0 Batch_no:  2 Loss :=== 10.788266       Accuracy:== 0.0
Epoch:  0 Batch_no:  3 Loss :=== 11.447231       Accuracy:== 0.0
Epoch:  0 Batch_no:  4 Loss :=== 8.50559       Accuracy:== 0.02
Epoch:  0 Batch_no:  5 Loss :=== 7.7112927       Accuracy:== 0.02
Epoch:  0 Batch_no:  6 Loss :=== 6.9052477       Accuracy:== 0.0
Epoch:  0 Batch_no:  7 Loss :=== 5.8406134       Accuracy:== 0.0
Epoch:  0 Batch_no:  8 Loss :=== 6.125981       Accuracy:== 0.0
Epoch:  0 Batch_no:  9 Loss :=== 5.658981       Accuracy:== 0.06
Epoch:  0 Batch_no:  10 Loss :=== 6.099097       Accuracy:== 0.0
Epoch:  0 Batch_no:  11 Loss :=== 5.537284       Accuracy:== 0.0
Epoch:  0 Batch_no:  12 Loss :=== 5.113553       Accuracy:== 0.0
Epoch:  0 Batch_no:  13 Loss :=== 4.9650936       Accuracy:== 0.02
Epoch:  0 Batch_no:  14 Loss :=== 5.0673366       Accuracy:== 0.0
Epoch:  0 Batch_no:  15 Loss :=== 5.0113444       Accuracy:== 0.0
Epoch:  0 Batch_no:  1

Epoch:  0 Batch_no:  126 Loss :=== 4.787584       Accuracy:== 0.02
Epoch:  0 Batch_no:  127 Loss :=== 4.589793       Accuracy:== 0.02
Epoch:  0 Batch_no:  128 Loss :=== 4.6373696       Accuracy:== 0.02
Epoch:  0 Batch_no:  129 Loss :=== 4.7436156       Accuracy:== 0.04
Epoch:  0 Batch_no:  130 Loss :=== 4.721532       Accuracy:== 0.02
Epoch:  0 Batch_no:  131 Loss :=== 4.731496       Accuracy:== 0.0
Epoch:  0 Batch_no:  132 Loss :=== 4.6012087       Accuracy:== 0.0
Epoch:  0 Batch_no:  133 Loss :=== 4.731957       Accuracy:== 0.0
Epoch:  0 Batch_no:  134 Loss :=== 4.6718116       Accuracy:== 0.02
Epoch:  0 Batch_no:  135 Loss :=== 4.735535       Accuracy:== 0.02
Epoch:  0 Batch_no:  136 Loss :=== 4.5949044       Accuracy:== 0.04
Epoch:  0 Batch_no:  137 Loss :=== 4.8329854       Accuracy:== 0.02
Epoch:  0 Batch_no:  138 Loss :=== 4.469835       Accuracy:== 0.06
Epoch:  0 Batch_no:  139 Loss :=== 4.7238135       Accuracy:== 0.02
Epoch:  0 Batch_no:  140 Loss :=== 4.532405       Accuracy

Epoch:  1 Batch_no:  248 Loss :=== 4.585377       Accuracy:== 0.04
Epoch:  1 Batch_no:  249 Loss :=== 4.5862107       Accuracy:== 0.08
Epoch:  1 Batch_no:  250 Loss :=== 4.5060596       Accuracy:== 0.0
Epoch:  1 Batch_no:  251 Loss :=== 4.50368       Accuracy:== 0.02
Epoch:  1 Batch_no:  252 Loss :=== 4.469641       Accuracy:== 0.02
Epoch:  1 Batch_no:  253 Loss :=== 4.5989094       Accuracy:== 0.04
Epoch:  1 Batch_no:  254 Loss :=== 4.418099       Accuracy:== 0.04
Epoch:  1 Batch_no:  255 Loss :=== 4.4579277       Accuracy:== 0.06
Epoch:  1 Batch_no:  256 Loss :=== 4.6434455       Accuracy:== 0.0
Epoch:  1 Batch_no:  257 Loss :=== 4.493144       Accuracy:== 0.06
Epoch:  1 Batch_no:  258 Loss :=== 4.3956656       Accuracy:== 0.06
Epoch:  1 Batch_no:  259 Loss :=== 4.469254       Accuracy:== 0.04
Epoch:  1 Batch_no:  260 Loss :=== 4.4355545       Accuracy:== 0.0
Epoch:  1 Batch_no:  261 Loss :=== 4.3706164       Accuracy:== 0.02
Epoch:  1 Batch_no:  262 Loss :=== 4.387688       Accuracy

Epoch:  1 Batch_no:  370 Loss :=== 4.2571473       Accuracy:== 0.04
Epoch:  1 Batch_no:  371 Loss :=== 4.5008216       Accuracy:== 0.04
Epoch:  1 Batch_no:  372 Loss :=== 4.4166517       Accuracy:== 0.04
Epoch:  1 Batch_no:  373 Loss :=== 4.220476       Accuracy:== 0.14
Epoch:  1 Batch_no:  374 Loss :=== 4.565719       Accuracy:== 0.02
Epoch:  1 Batch_no:  375 Loss :=== 4.3220067       Accuracy:== 0.06
Epoch:  1 Batch_no:  376 Loss :=== 4.221592       Accuracy:== 0.08
Epoch:  1 Batch_no:  377 Loss :=== 4.0728908       Accuracy:== 0.08
Epoch:  1 Batch_no:  378 Loss :=== 4.3672414       Accuracy:== 0.02
Epoch:  1 Batch_no:  379 Loss :=== 4.2320704       Accuracy:== 0.02
Epoch:  1 Batch_no:  380 Loss :=== 4.0797763       Accuracy:== 0.02
Epoch:  1 Batch_no:  381 Loss :=== 4.370276       Accuracy:== 0.06
Epoch:  1 Batch_no:  382 Loss :=== 4.3462873       Accuracy:== 0.08
Epoch:  1 Batch_no:  383 Loss :=== 4.3702607       Accuracy:== 0.12
Epoch:  1 Batch_no:  384 Loss :=== 4.440514       Ac

Epoch:  2 Batch_no:  492 Loss :=== 4.21739       Accuracy:== 0.12
Epoch:  2 Batch_no:  493 Loss :=== 3.8760805       Accuracy:== 0.12
Epoch:  2 Batch_no:  494 Loss :=== 4.1154113       Accuracy:== 0.06
Epoch:  2 Batch_no:  495 Loss :=== 3.9851606       Accuracy:== 0.08
Epoch:  2 Batch_no:  496 Loss :=== 4.0795546       Accuracy:== 0.08
Epoch:  2 Batch_no:  497 Loss :=== 4.121037       Accuracy:== 0.08
Epoch:  2 Batch_no:  498 Loss :=== 4.2179832       Accuracy:== 0.2
Epoch:  2 Batch_no:  499 Loss :=== 4.358952       Accuracy:== 0.0
Epoch:  2 Batch_no:  500 Loss :=== 4.241235       Accuracy:== 0.04
Epoch:  2 Batch_no:  501 Loss :=== 4.1479816       Accuracy:== 0.02
Epoch:  2 Batch_no:  502 Loss :=== 4.11235       Accuracy:== 0.04
Epoch:  2 Batch_no:  503 Loss :=== 4.142113       Accuracy:== 0.08
Epoch:  2 Batch_no:  504 Loss :=== 4.115287       Accuracy:== 0.1
Epoch:  2 Batch_no:  505 Loss :=== 4.0463715       Accuracy:== 0.1
Epoch:  2 Batch_no:  506 Loss :=== 4.2604203       Accuracy:=

...

Epoch:  7 Batch_no:  1458 Loss :=== 0.24718945       Accuracy:== 0.94
Epoch:  7 Batch_no:  1459 Loss :=== 0.15510923       Accuracy:== 0.96
Epoch:  7 Batch_no:  1460 Loss :=== 0.17296688       Accuracy:== 0.92
Epoch:  7 Batch_no:  1461 Loss :=== 0.41920483       Accuracy:== 0.86
Epoch:  7 Batch_no:  1462 Loss :=== 0.18826939       Accuracy:== 0.92
Epoch:  7 Batch_no:  1463 Loss :=== 0.26039502       Accuracy:== 0.94
Epoch:  7 Batch_no:  1464 Loss :=== 0.08283558       Accuracy:== 1.0
Epoch:  7 Batch_no:  1465 Loss :=== 0.27932948       Accuracy:== 0.88
Epoch:  7 Batch_no:  1466 Loss :=== 0.32722402       Accuracy:== 0.88
Epoch:  7 Batch_no:  1467 Loss :=== 0.3901651       Accuracy:== 0.9
Epoch:  7 Batch_no:  1468 Loss :=== 0.13756897       Accuracy:== 0.88
Epoch:  7 Batch_no:  1469 Loss :=== 0.2527766       Accuracy:== 0.9
Epoch:  7 Batch_no:  1470 Loss :=== 0.23321413       Accuracy:== 0.9
Epoch:  7 Batch_no:  1471 Loss :=== 0.055414658       Accuracy:== 1.0
Epoch:  7 Batch_no:  1472 

Epoch:  7 Batch_no:  1577 Loss :=== 0.5520864       Accuracy:== 0.82
Epoch:  7 Batch_no:  1578 Loss :=== 0.34870026       Accuracy:== 0.9
Epoch:  7 Batch_no:  1579 Loss :=== 0.33595714       Accuracy:== 0.8
Epoch:  7 Batch_no:  1580 Loss :=== 0.45310348       Accuracy:== 0.88
Epoch:  7 Batch_no:  1581 Loss :=== 0.13055988       Accuracy:== 0.96
Epoch:  7 Batch_no:  1582 Loss :=== 0.3905027       Accuracy:== 0.9
Epoch:  7 Batch_no:  1583 Loss :=== 0.31284314       Accuracy:== 0.9
Epoch:  7 Batch_no:  1584 Loss :=== 0.18995257       Accuracy:== 0.96
Epoch:  7 Batch_no:  1585 Loss :=== 0.23223436       Accuracy:== 0.94
Epoch:  7 Batch_no:  1586 Loss :=== 0.6790191       Accuracy:== 0.88
Epoch:  7 Batch_no:  1587 Loss :=== 0.3034088       Accuracy:== 0.9
Epoch:  7 Batch_no:  1588 Loss :=== 0.45391193       Accuracy:== 0.84
Epoch:  7 Batch_no:  1589 Loss :=== 0.12996775       Accuracy:== 0.92
Epoch:  7 Batch_no:  1590 Loss :=== 0.55183107       Accuracy:== 0.86
Epoch:  7 Batch_no:  1591 Los

Epoch:  8 Batch_no:  1695 Loss :=== 0.15154548       Accuracy:== 0.92
Epoch:  8 Batch_no:  1696 Loss :=== 0.064645946       Accuracy:== 0.98
Epoch:  8 Batch_no:  1697 Loss :=== 0.5746575       Accuracy:== 0.9
Epoch:  8 Batch_no:  1698 Loss :=== 0.3000805       Accuracy:== 0.9
Epoch:  8 Batch_no:  1699 Loss :=== 0.24124359       Accuracy:== 0.94
Epoch:  8 Batch_no:  1700 Loss :=== 0.14844297       Accuracy:== 0.92
Epoch:  8 Batch_no:  1701 Loss :=== 0.103966914       Accuracy:== 0.98
Epoch:  8 Batch_no:  1702 Loss :=== 0.4773802       Accuracy:== 0.94
Epoch:  8 Batch_no:  1703 Loss :=== 0.16675255       Accuracy:== 0.96
Epoch:  8 Batch_no:  1704 Loss :=== 0.3902702       Accuracy:== 0.9
Epoch:  8 Batch_no:  1705 Loss :=== 0.16315733       Accuracy:== 0.92
Epoch:  8 Batch_no:  1706 Loss :=== 0.36819988       Accuracy:== 0.92
Epoch:  8 Batch_no:  1707 Loss :=== 0.42550504       Accuracy:== 0.9
Epoch:  8 Batch_no:  1708 Loss :=== 0.37445855       Accuracy:== 0.9
Epoch:  8 Batch_no:  1709 L

Epoch:  9 Batch_no:  1813 Loss :=== 0.25944048       Accuracy:== 0.98
Epoch:  9 Batch_no:  1814 Loss :=== 0.06731671       Accuracy:== 0.98
Epoch:  9 Batch_no:  1815 Loss :=== 0.3245132       Accuracy:== 0.92
Epoch:  9 Batch_no:  1816 Loss :=== 0.07880921       Accuracy:== 0.98
Epoch:  9 Batch_no:  1817 Loss :=== 0.11413373       Accuracy:== 0.94
Epoch:  9 Batch_no:  1818 Loss :=== 0.596534       Accuracy:== 0.92
Epoch:  9 Batch_no:  1819 Loss :=== 0.4925714       Accuracy:== 0.86
Epoch:  9 Batch_no:  1820 Loss :=== 0.160325       Accuracy:== 0.92
Epoch:  9 Batch_no:  1821 Loss :=== 0.58869976       Accuracy:== 0.88
Epoch:  9 Batch_no:  1822 Loss :=== 0.24329214       Accuracy:== 0.9
Epoch:  9 Batch_no:  1823 Loss :=== 0.28786942       Accuracy:== 0.88
Epoch:  9 Batch_no:  1824 Loss :=== 0.23361723       Accuracy:== 0.94
Epoch:  9 Batch_no:  1825 Loss :=== 0.067582674       Accuracy:== 0.98
Epoch:  9 Batch_no:  1826 Loss :=== 0.08574761       Accuracy:== 0.96
Epoch:  9 Batch_no:  1827 

Epoch:  9 Batch_no:  1931 Loss :=== 0.7254688       Accuracy:== 0.92
Epoch:  9 Batch_no:  1932 Loss :=== 0.68902886       Accuracy:== 0.86
Epoch:  9 Batch_no:  1933 Loss :=== 0.5689866       Accuracy:== 0.9
Epoch:  9 Batch_no:  1934 Loss :=== 0.44359046       Accuracy:== 0.9
Epoch:  9 Batch_no:  1935 Loss :=== 0.39423618       Accuracy:== 0.9
Epoch:  9 Batch_no:  1936 Loss :=== 0.46235874       Accuracy:== 0.88
Epoch:  9 Batch_no:  1937 Loss :=== 0.7828582       Accuracy:== 0.82
Epoch:  9 Batch_no:  1938 Loss :=== 0.7312876       Accuracy:== 0.74
Epoch:  9 Batch_no:  1939 Loss :=== 0.23952444       Accuracy:== 0.94
Epoch:  9 Batch_no:  1940 Loss :=== 0.19435911       Accuracy:== 0.94
Epoch:  9 Batch_no:  1941 Loss :=== 0.63472044       Accuracy:== 0.82
Epoch:  9 Batch_no:  1942 Loss :=== 0.42933518       Accuracy:== 0.86
Epoch:  9 Batch_no:  1943 Loss :=== 0.40476018       Accuracy:== 0.88
Epoch:  9 Batch_no:  1944 Loss :=== 0.38354775       Accuracy:== 0.94
Epoch:  9 Batch_no:  1945 L